In [2]:
import requests
import os
import pandas as pd
from tqdm import tqdm

import dotenv
dotenv.load_dotenv()

def get_nutrients(ing_list, serving_size):
    """Obtiene datos nutricionales de una lista de ingredientes usando la API de EDAMAM.

    Esta función envía una lista de ingredientes a la API de EDAMAM para obtener información
    nutricional detallada para cada ingrediente y ajusta los datos según el tamaño de porción.

    Args:
        ing_list (list): Lista de ingredientes en formato de texto.
        serving_size (int): Tamaño de porción para ajustar los valores nutricionales.

    Returns:
        pd.DataFrame: DataFrame con los nutrientes de cada ingrediente, incluyendo:
            - 'Ingredient' (str): Nombre del ingrediente.
            - 'Weight (g)' (float): Peso en gramos del ingrediente ajustado al tamaño de porción.
            - 'Calories (kcal)' (float): Calorías por porción.
            - 'Protein (g)' (float): Proteína por porción.
            - 'Fat (g)' (float): Grasas por porción.
            - 'Carbohydrates (g)' (float): Carbohidratos por porción.
            - 'Sugar (g)' (float): Azúcar por porción.
            - 'Fiber (g)' (float): Fibra por porción.
            - 'Serving weight (g)' (float): Peso total ajustado al tamaño de porción.

    Raises:
        HTTPError: Si hay un problema en la solicitud a la API de EDAMAM.
    """
    url = "https://api.edamam.com/api/nutrition-details"


    headers = {
        "Content-Type": "application/json"
    }

    data = {
        "ingr": ing_list
    }

    response = requests.post(url, headers=headers, 
            params={"app_id": os.getenv('edamam_session_id'), "app_key": os.getenv('edamam_api_key')}, json=data)

    if response.status_code == 200:
        nutrition_data = response.json()
        nutrient_list = []
        for ingredient in nutrition_data.get("ingredients", []):
            try:
                 ingredient.get("parsed")[0] # Para aquellos ingredientes que la api no reconoce
            except:
                continue
            ingredient_name = ingredient.get("parsed")[0].get("foodMatch")
            nutrients = ingredient.get("parsed")[0].get("nutrients")
            weight = ingredient.get("parsed")[0].get("weight")
            total_weight = nutrition_data.get("totalWeight")

            calories = nutrients.get("ENERC_KCAL")["quantity"]
            protein = nutrients.get("PROCNT")["quantity"]
            fat = nutrients.get("FAT")["quantity"]
            carbs = nutrients.get("CHOCDF")["quantity"]
            try:
                fiber = nutrients.get("FIBTG")["quantity"]
            except:
                fiber = 0
            try:
                sugar = nutrients.get("SUGAR")["quantity"]
            except:
                sugar = 0
           
            nutrient_list.append({
                "Ingredient": ingredient_name,
                "Weight (g)" : weight/serving_size,
                "Calories (kcal)": calories/serving_size,
                "Protein (g)": protein/serving_size,
                "Fat (g)": fat/serving_size,
                "Carbohydrates (g)": carbs/serving_size,
                "Sugar (g)" : sugar/serving_size,
                "Fiber (g)" : fiber/serving_size,
                "Serving weight (g)" : total_weight/serving_size
            })
        nutrients_df = pd.DataFrame(nutrient_list)
        return nutrients_df

    else:
        print("Error:", response.status_code, response.json())
        return None

In [3]:
from deep_translator import GoogleTranslator

def translate_es_en(text):
    return GoogleTranslator(source='es', target='en').translate(text)

In [5]:
recipes = pd.read_json(r'recetas_scrapper\recetas_dap_no_delay.jsonl', lines=True)
for i in range(len(recipes)):
    ingredients = recipes["ingredientes"].loc[i]
    resultados = [
        f"{n} {c} {u}".strip() if u else f"{n} {c}".strip()
        for n, c, u in zip(ingredients['nombre'], ingredients['cantidad'], ingredients['unidad'])
        if c is not None
    ]
    en_ingredients = [translate_es_en(e) for e in resultados]
    print(en_ingredients)



['Peeled and seeded melon 1.0', 'Liquid cream for cooking 100.0', 'Iberian ham cut into strips 120.0 g']
['Peeled and chopped onion 1.0', 'Peeled and chopped carrot 3.0', 'Extra virgin olive oil 100.0 g', 'Egg 3.0', 'Garlic clove 3.0', 'Basmati rice 350.0 g', 'Water 800.0 ml']
['Fresh salmon, skinless, in loin 4.0', 'Lime zest and a little juice (for the marinade) 1.0', 'Tomato (for serving) 2.0', 'Red onion (to serve) 1.0', 'Greek yogurt (for serving) 125.0 g']
['Bacon slices 200.0 g', 'Shallot 10.0', 'Mushrooms 200.0 g', 'Veal shank 1.2 kg', 'Wheat flour 30.0 g', 'Red wine 750.0 ml', 'Crushed tomato 400.0 g', 'Beef broth 400.0 ml', 'Fresh thyme on branch 2.0', 'Laurel leaf 2.0']
['Razors 16.0', 'Chinese chives or fresh garlic 30.0 g', 'Lemon juice 8.0 ml']
['Butter at room temperature 225.0 g', 'Sugar 125.0 g', 'Apple 1.0', 'Chopped walnuts 50.0 g', 'Wheat flour 300.0 g', 'Egg 1.0']
['Pear tomato 700.0 g', 'Hard bread 60.0 g']
['Butter at room temperature 100.0 g', 'Sugar 125.0 g', '

KeyboardInterrupt: 

In [42]:
en_ingredients = [translate_es_en(e) for e in resultados]

In [45]:
resultados

['Frutos del bosque congelados 400.0 g',
 'Nata líquida para cocinar 200.0 ml',
 'Azúcar 100.0 g',
 'Azúcar invertido 50.0 g',
 'Azúcar glasé para montar la nata 100.0 g']

In [44]:
en_ingredients

['Frozen forest fruits 400.0 g',
 'Liquid cream for cooking 200.0 ml',
 'Sugar 100.0 g',
 'Inverted sugar 50.0 g',
 'Icing sugar for whipping the cream 100.0 g']

In [47]:
df_final = get_nutrients(en_ingredients, 6)

In [ ]:
df_final["title"] = "Receta1"

In [52]:
df_recipes = df_final.groupby('title').agg({
    'Calories (kcal)': 'sum',
    'Protein (g)': 'sum',
    'Fat (g)': 'sum',
    'Carbohydrates (g)': 'sum',
    'Sugar (g)': 'sum',
    'Fiber (g)': 'sum',
    'Serving weight (g)': 'first',  # Asumimos que el peso por porción es el mismo para toda la receta
}).reset_index()

In [53]:
df_recipes

,title,Calories (kcal),Protein (g),Fat (g),Carbohydrates (g),Sugar (g),Fiber (g),Serving weight (g)
0,Receta1,295.784559,1.187249,12.17215,47.833916,42.230787,0.666667,141.921929
